In [1]:
import os
import json
import pandas as pd
import traceback

In [ ]:
from dotenv import load_dotenv
load_dotenv()
KEY=os.getenv('OPENAI_API_KEY')
print(KEY)

In [5]:
from langchain.chat_models import ChatOpenAI

llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [7]:
RESPONSE_JSON={
    "1": {
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct_answer",
    },
    "2": {
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct_answer",
    },
    "1": {
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct_answer",
    },
}

TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
"create a quiz of {number} multiple choice questions for {topic} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well. 
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
###RESPONSE_JSON
{response_json}
"""

In [17]:
quiz_prompt = PromptTemplate(
    input_variables=['text', 'topic', 'number', 'tone', 'response_json'],
    template=TEMPLATE
)

In [18]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_prompt, output_key="quiz", verbose=True)

In [8]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {topic} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity\
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}
Check from an expert English Writer of the above quiz:
"""

In [19]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["topic", "quiz"], template=TEMPLATE2)
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review",verbose=True)

In [24]:
gen_eval_chain = SequentialChain(chains=[quiz_chain, review_chain], 
                                 input_variables=['text', 'topic','number', 'tone', 'response_json'],
                                 output_variables=["quiz","review"],
                                 verbose=True)

In [21]:
file_path='/Users/prasadrg/data/projects/mcqgen/data.txt'
with open(file_path, 'r') as file:
    TEXT=file.read()

print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]
Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contributed 

In [22]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}, "2": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}}'

In [25]:
TOPIC="AI"
NUMBER=5
TONE="simple"

with get_openai_callback() as cb:
    response=gen_eval_chain(
    {
    "text": TEXT,
    "topic": TOPIC,
    "number": NUMBER, 
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON)
    }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]
Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used b

In [26]:
print(f"Total tokens: {cb.total_tokens}")
print(f"Prompt tokens: {cb.prompt_tokens}")
print(f"Completion tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total tokens: 1799
Prompt tokens: 1342
Completion tokens: 457
Total Cost: 0.002927


In [27]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems 

In [28]:
quiz = json.loads(response.get('quiz'))
quiz

{'1': {'mcq': 'Who coined the term machine learning in 1959?',
  'options': {'a': 'Donald Hebb',
   'b': 'Walter Pitts',
   'c': 'Arthur Samuel',
   'd': 'Raytheon Company'},
  'correct': 'c'},
 '2': {'mcq': 'What was the early machine learning model introduced by Arthur Samuel in the 1950s?',
  'options': {'a': 'Program for calculating winning chance in checkers',
   'b': 'Program for analyzing sonar signals',
   'c': 'Program for recognizing speech patterns',
   'd': 'Program for studying human cognitive processes'},
  'correct': 'a'},
 '3': {'mcq': 'Which book introduced a theoretical neural structure by Donald Hebb in 1949?',
  'options': {'a': 'Learning Machines by Nilsson',
   'b': 'Computing Machinery and Intelligence by Alan Turing',
   'c': 'The Organization of Behavior by Donald Hebb',
   'd': 'The Organization of Artificial Neurons by Warren McCulloch'},
  'correct': 'c'},
 '4': {'mcq': "What was the purpose of the experimental 'learning machine' Cybertron developed by Rayth

In [29]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

quiz_table_data

[{'MCQ': 'Who coined the term machine learning in 1959?',
  'Choices': 'a: Donald Hebb | b: Walter Pitts | c: Arthur Samuel | d: Raytheon Company',
  'Correct': 'c'},
 {'MCQ': 'What was the early machine learning model introduced by Arthur Samuel in the 1950s?',
  'Choices': 'a: Program for calculating winning chance in checkers | b: Program for analyzing sonar signals | c: Program for recognizing speech patterns | d: Program for studying human cognitive processes',
  'Correct': 'a'},
 {'MCQ': 'Which book introduced a theoretical neural structure by Donald Hebb in 1949?',
  'Choices': 'a: Learning Machines by Nilsson | b: Computing Machinery and Intelligence by Alan Turing | c: The Organization of Behavior by Donald Hebb | d: The Organization of Artificial Neurons by Warren McCulloch',
  'Correct': 'c'},
 {'MCQ': "What was the purpose of the experimental 'learning machine' Cybertron developed by Raytheon Company in the early 1960s?",
  'Choices': 'a: Analyze electrocardiograms | b: Rec

In [30]:
quiz_df=pd.DataFrame(quiz_table_data)
quiz_df

,MCQ,Choices,Correct
0,Who coined the term machine learning in 1959?,a: Donald Hebb | b: Walter Pitts | c: Arthur S...,c
1,What was the early machine learning model intr...,a: Program for calculating winning chance in c...,a
2,Which book introduced a theoretical neural str...,a: Learning Machines by Nilsson | b: Computing...,c
3,What was the purpose of the experimental 'lear...,a: Analyze electrocardiograms | b: Recognize p...,b
4,"According to Tom M. Mitchell, what does a comp...",a: Improve performance at tasks in T with expe...,a


In [32]:
quiz_df.to_csv("machinelearning.csv",index=False)